In [1]:
import pandas as pd

In [2]:
acc_col = ['Recording timestamp', 'Computer timestamp [μs]', 'Accelerometer X [m/s²]', 'Accelerometer Y [m/s²]', 'Accelerometer Z [m/s²]']
acc_cols = ['Accelerometer X [m/s²]', 'Accelerometer Y [m/s²]', 'Accelerometer Z [m/s²]']

et_col = ['Recording timestamp', 'Computer timestamp [μs]', 'Gaze point X [MCS px]', 'Gaze point Y [MCS px]','Gaze point 3D X [HUCS mm]', 'Gaze point 3D Y [HUCS mm]','Gaze point 3D Z [HUCS mm]', 'Gaze direction left X [HUCS norm]','Gaze direction left Y [HUCS norm]','Gaze direction left Z [HUCS norm]','Gaze direction right X [HUCS norm]','Gaze direction right Y [HUCS norm]','Gaze direction right Z [HUCS norm]', 'Pupil position left X [HUCS mm]','Pupil position left Y [HUCS mm]', 'Pupil position left Z [HUCS mm]','Pupil position right X [HUCS mm]', 'Pupil position right Y [HUCS mm]','Pupil position right Z [HUCS mm]', 'Pupil diameter left [mm]','Pupil diameter right [mm]', 'Eye movement type','Gaze event duration [ms]', 'Fixation point X [MCS px]','Fixation point Y [MCS px]']
etcol = ['Recording timestamp', 'Computer timestamp [μs]', 'Gaze point X [MCS px]', 'Gaze point Y [MCS px]','Gaze point 3D X [HUCS mm]', 'Gaze point 3D Y [HUCS mm]','Gaze point 3D Z [HUCS mm]', 'Gaze direction left X [HUCS norm]','Gaze direction left Y [HUCS norm]','Gaze direction left Z [HUCS norm]','Gaze direction right X [HUCS norm]','Gaze direction right Y [HUCS norm]','Gaze direction right Z [HUCS norm]', 'Pupil position left X [HUCS mm]','Pupil position left Y [HUCS mm]', 'Pupil position left Z [HUCS mm]','Pupil position right X [HUCS mm]', 'Pupil position right Y [HUCS mm]','Pupil position right Z [HUCS mm]', 'Pupil diameter left [mm]','Pupil diameter right [mm]', 'Fixation point X [MCS px]','Fixation point Y [MCS px]']
et_cols = ['Gaze point X [MCS px]', 'Gaze point Y [MCS px]','Gaze point 3D X [HUCS mm]', 'Gaze point 3D Y [HUCS mm]','Gaze point 3D Z [HUCS mm]', 'Gaze direction left X [HUCS norm]','Gaze direction left Y [HUCS norm]','Gaze direction left Z [HUCS norm]','Gaze direction right X [HUCS norm]','Gaze direction right Y [HUCS norm]','Gaze direction right Z [HUCS norm]', 'Pupil position left X [HUCS mm]','Pupil position left Y [HUCS mm]', 'Pupil position left Z [HUCS mm]','Pupil position right X [HUCS mm]', 'Pupil position right Y [HUCS mm]','Pupil position right Z [HUCS mm]', 'Pupil diameter left [mm]','Pupil diameter right [mm]', 'Fixation point X [MCS px]','Fixation point Y [MCS px]']

gy_col = ['Recording timestamp', 'Computer timestamp [μs]', 'Gyro X [°/s]','Gyro Y [°/s]', 'Gyro Z [°/s]']
gy_cols = ['Gyro X [°/s]','Gyro Y [°/s]', 'Gyro Z [°/s]']


In [3]:
files = ['sub01', 'sub02', 'sub03', 'sub04', 'sub05', 'sub06', 'sub07', 'sub08', 'sub09', 'sub10', 'sub11', 'sub12', 'sub13', 'sub14', 'sub15', 'sub16', 'sub17', 'sub18', 'sub19', 'sub20', 'sub21', 'sub22', 'sub23', 'sub24', 'sub25', 'sub26', 'sub27', 'sub28', 'sub29', 'sub30', 'sub31', 'sub32']
subfiles = ['{}.csv'.format(x) for x in range(5,17)]
skip_files = ['../data/cleaned_driving_data/sub01/13.csv', '../data/cleaned_driving_data/sub03/9.csv']


In [4]:
tot = pd.read_csv('../data/timestamp_data/TO_micros.csv')

for i in range(len(files)):
    
    acc = pd.read_csv('../data/tobii_acc_data/'+files[i]+'.csv')
    acc.columns = acc_col
    acc = acc.groupby(acc.index // 10).mean()
    et = pd.read_csv('../data/tobii_et_data/'+files[i]+'.csv')
    et.columns = et_col
    et = et[etcol]
    et = et.groupby(et.index // 10).mean()
    gy = pd.read_csv('../data/tobii_gy_data/'+files[i]+'.csv')
    gy.columns = gy_col
    gy = gy.groupby(gy.index // 10).mean()

    for j in range(len(subfiles)):
        
        path = '../data/cleaned_driving_data/'+files[i]+'/'+subfiles[j]
        
        if path in skip_files:
            continue
        
        df = pd.read_csv('../data/cleaned_driving_data/'+files[i]+'/'+subfiles[j])

        to_idx = df[df['longitudinal_velocity'] != 88].index.values[0]
        
        acc_idx = acc[acc['Recording timestamp'] <= tot.iloc[j,i]].index.values[-1]
        et_idx = et[et['Recording timestamp'] <= tot.iloc[j,i]].index.values[-1]
        gy_idx = gy[gy['Recording timestamp'] <= tot.iloc[j,i]].index.values[-1]

        lacc_idx = acc_idx - to_idx
        let_idx = et_idx - to_idx
        lgy_idx = gy_idx - to_idx

        uacc_idx = acc_idx + (len(df) - to_idx)
        uet_idx = et_idx + (len(df) - to_idx)
        ugy_idx = gy_idx + (len(df) - to_idx)

        acc_t = acc[acc_cols][lacc_idx:uacc_idx].reset_index(drop=True)
        et_t = et[et_cols][let_idx:uet_idx].reset_index(drop=True)
        gy_t = gy[gy_cols][lgy_idx:ugy_idx].reset_index(drop=True)

        df = pd.concat([df, acc_t, et_t, gy_t], axis=1)
        df.to_csv('../data/cleaned_driving_data_tobii/'+files[i]+'/'+subfiles[j], index=False)
